In [ ]:
import warnings

# Ignore warnings
warnings.filterwarnings("ignore")

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# !unzip "/content/drive/MyDrive/Dataset/data.zip" -d "/content/data"
# !unzip "/content/drive/MyDrive/Dataset/data.zip"

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
# # COLAB
# train_dir = 'train/'
# validation_dir = 'validation/'
# test_dir = 'test/'

# KAGGLE
train_dir = '/kaggle/input/bangla-sign/train/'
validation_dir = '/kaggle/input/bangla-sign/validation/'
test_dir = '/kaggle/input/bangla-sign/test/'

In [ ]:
# Data augmentation and normalization
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(224, 224),
                                                    batch_size=32,
                                                    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                        target_size=(224, 224),
                                                        batch_size=32,
                                                        class_mode='categorical')

In [ ]:
# Define base models
def create_base_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dropout(0.5),
        Dense(512, activation='relu'),
        Dense(38, activation='softmax')
    ])
    return model

num_models = 3
base_models = [create_base_model() for _ in range(num_models)]

In [ ]:
# Train each base model
for i, model in enumerate(base_models):
    print(f"Training Base Model {i+1}")
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(train_generator, epochs=10, validation_data=validation_generator)

In [ ]:
# Combine the predictions of the base models
def ensemble_predict(models, generator):
    preds = [model.predict(generator) for model in models]
    return np.mean(preds, axis=0)

ensemble_preds_validation = ensemble_predict(base_models, validation_generator)

In [ ]:
# Evaluate the ensemble model
ensemble_accuracy = np.mean(np.argmax(ensemble_preds_validation, axis=1) == validation_generator.classes)
print("Ensemble Accuracy:", ensemble_accuracy)